In [1]:
import os
from tqdm import tqdm
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
pd.set_option('display.float_format', '{:.3f}'.format)
x_min, x_max = -1., 1.
y_min, y_max = -.3, .3

In [2]:
trn_index_list = np.load("../input/eyn-original/trn_index_list.npy", allow_pickle=True)
val_index_list = np.load("../input/eyn-original/val_index_list.npy", allow_pickle=True)
print([np.shape(trn_index_list[i]) for i in range(np.shape(trn_index_list)[0])])
print([np.shape(val_index_list[i]) for i in range(np.shape(val_index_list)[0])])
train_targets_inside = np.load("../input/eyn-original/train_targets_inside.npy")

[(100547,), (100547,), (100547,), (100548,)]
[(33516,), (33516,), (33516,), (33515,)]


In [3]:
df_train = pd.read_pickle("../input/eyn-pre-unravel-df/df_train.pickle")
df_test = pd.read_pickle("../input/eyn-pre-unravel-df/df_test.pickle")
print(df_train.shape, df_test.shape)

(2815323, 23) (703815, 23)


In [4]:
# cat_col = ['entry_in', 'exit_in', 'clus_entry', 'clus_exit', 'tid_0', 'tid_1']
cat_col = ['entry_in', 'exit_in', 'tid_0', 'tid_1']
df_train[cat_col] = df_train[cat_col].astype('category')
df_test[cat_col] = df_test[cat_col].astype('category')

In [5]:
drop_col = (['x_exit_0', 'y_exit_0', 'vmean_0', 'vmax_0', 'vmin_0', 
             'exit_in_0', 'dist_0', 'speed_0', 'dir_x_0', 'dir_y_0', 'clus_exit_0'] + 
            ['clus_entry_'+ str(i) for i in range(21)] + 
            ['clus_exit_'+ str(i) for i in range(1,21)])
drop_col = []  # not dropping any columns

In [6]:
# coding: utf-8
# pylint: disable = invalid-name, C0111
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, log_loss
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score

def scoring_package(y_true, y_pred):
    y_pred_class = np.array([k > 0.5 for k in y_pred])
    f1 = f1_score(y_true, y_pred_class)
    precision = precision_score(y_true, y_pred_class)
    recall = recall_score(y_true, y_pred_class)
    roc = roc_auc_score(y_true, y_pred)
    return f1, precision, recall, roc

target = np.load("../input/eyn-original/train_targets.npy")
y_label_x = target[:, 0]
y_label_y = target[:, 1]

binary = np.load("../input/eyn-original/train_targets_inside.npy")

In [7]:
trn_embedding = np.load("../input/eynembedding/train_lstm.npy")
test_embedding = np.load("../input/eynembedding/test_lstm.npy")

In [8]:
import math
time_dummy = []
for i in df_train["t_entry"]:
    entry = np.repeat(0, 16)
    if math.isnan(i):
        time_dummy.append([np.nan]*16)
    else:
        num = int((i + 1.5) // 0.1)
        if num != 0:
            entry[num-1] = 1
        time_dummy.append(entry)
        
time_dummy = np.array(time_dummy)
# df_train_unstack = df_train.unstack()
# unstack_col_names = ["_".join([tup[0],str(tup[1])]) for tup in df_train_unstack.columns.values]
# df_train_unstack.columns = unstack_col_names
# df_train_unstack = df_train_unstack.drop(drop_col, axis=1)
# df_train_unstack.head()
df_train_unstack = df_train.values
df_train_unstack = np.concatenate([df_train_unstack, time_dummy], axis=1)
# df_train_unstack = df_train_unstack.reshape((int(df_train_unstack.shape[0]/21), 43*21))
df_train_unstack = df_train_unstack.reshape((int(df_train_unstack.shape[0]/21), 39*21))
df_train_unstack = np.concatenate([df_train_unstack, trn_embedding], axis=1)

import math
time_dummy = []
for i in df_test["t_entry"]:
    entry = np.repeat(0, 16)
    if math.isnan(i):
        time_dummy.append([np.nan]*16)
    else:
        num = int((i + 1.5) // 0.1)
        if num != 0:
            entry[num-1] = 1
        time_dummy.append(entry)
        
df_test[cat_col] = df_test[cat_col].astype('category')
# df_test_unstack = df_test.values
# df_test_unstack = np.concatenate([df_test_unstack, time_dummy], axis=1)
# df_test_unstack = df_test_unstack.reshape((int(df_test_unstack.shape[0]/21), 39*21))
# df_test_unstack = df_test.unstack()
# df_test_unstack.columns = unstack_col_names
# df_test_unstack = df_test_unstack.drop(drop_col, axis=1)
# df_test_unstack = df_test_unstack.values
# df_test_unstack = np.concatenate([df_test_unstack, test_embedding], axis=1)

In [9]:
kf = KFold(n_splits=10, random_state=0)
indexes = kf.split(df_train_unstack)

In [10]:
num_boost_round = 300

valid_ = []
test_pred = []
for trn_index, val_index in indexes:
    # inputs: train_pd, train_targets_inside, and test_pd
    # probably try https://lightgbm.readthedocs.io/en/latest/Python-API.html#lightgbm.cv, but cluster complications
    y_pred_full_x = []
    test_preds = []

    for fold_num, i in enumerate(range(10)):
        x_trn = df_train_unstack
        binary_trn = binary

        # create dataset for lightgbm
        lgb_train_x = lgb.Dataset(x_trn, binary_trn)
        #     lgb_train_y = lgb.Dataset(x_trn, y_trn_y)
        #     lgb_eval_y = lgb.Dataset(x_val, y_val_y, reference=lgb_train_y)
        params = {
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'binary_logloss',
        'num_leaves': 63,
        'learning_rate': 0.05,  # dynamic one below
        'feature_fraction': 0.9,
        'bagging_fraction': 0.9,
        'bagging_freq': 5,
        'verbose': 0,
        'lambda': 0.1,
        'num_threads': 3,
        'seed': 42+fold_num
        }

        print("Training fold {}, hash: {}".format(fold_num, np.sum(val_index)%999))

        x_trn = df_train_unstack[trn_index]
        x_val = df_train_unstack[val_index]

        binary_val = binary[val_index]
        binary_trn = binary[trn_index]

        # create dataset for lightgbm
        lgb_train_x = lgb.Dataset(x_trn, binary_trn)
        lgb_eval_x = lgb.Dataset(x_val, binary_val, reference=lgb_train_x)
        #     lgb_train_y = lgb.Dataset(x_trn, y_trn_y)
        #     lgb_eval_y = lgb.Dataset(x_val, y_val_y, reference=lgb_train_y)

        gbm_x = lgb.train(params,
                        lgb_train_x,
                        valid_sets=[lgb_train_x, lgb_eval_x],
                        num_boost_round=num_boost_round,
                        early_stopping_rounds=20,
                        learning_rates=lambda iter: 0.1 * (0.995 ** iter),
                        verbose_eval=100)

        y_pred_x = gbm_x.predict(x_val, num_iteration=gbm_x.best_iteration)
        y_pred_full_x.append(y_pred_x)

        df_test_unstack = df_test.values
        df_test_unstack = np.concatenate([df_test_unstack, time_dummy], axis=1)
        df_test_unstack = df_test_unstack.reshape((int(df_test_unstack.shape[0]/21), 39*21))
        df_test_unstack = np.concatenate([df_test_unstack, test_embedding], axis=1)

        test_pred_x = gbm_x.predict(df_test_unstack, num_iteration=gbm_x.best_iteration).reshape(df_test_unstack.shape[0], 1)
        #     test_pred_y = gbm_y.predict(df_test_unstack, num_iteration=gbm_y.best_iteration).reshape(df_test_unstack.shape[0], 1)
        #     test_preds.append(np.concatenate([test_pred_x, test_pred_y], axis=1))
        test_preds.append(test_pred_x)
    pred_cate = np.average(test_preds, axis=0)
    y_pred_full_x = np.average(y_pred_full_x, axis=0)
    valid_,append(y_pred_full_x)
    test_pred.append(pred_cate)
np.save("pred_cate.npy", test_pred)
np.save("valid_pred.npy", valid_)

Training fold 0, hash: 78
Training until validation scores don't improve for 20 rounds.
[100]	training's binary_logloss: 0.104995	valid_1's binary_logloss: 0.14024
Early stopping, best iteration is:
[108]	training's binary_logloss: 0.103266	valid_1's binary_logloss: 0.140148
Training fold 1, hash: 78
Training until validation scores don't improve for 20 rounds.
[100]	training's binary_logloss: 0.105393	valid_1's binary_logloss: 0.140155
Early stopping, best iteration is:
[120]	training's binary_logloss: 0.10106	valid_1's binary_logloss: 0.139896
Training fold 2, hash: 78
Training until validation scores don't improve for 20 rounds.
[100]	training's binary_logloss: 0.105496	valid_1's binary_logloss: 0.140022
Early stopping, best iteration is:
[85]	training's binary_logloss: 0.109333	valid_1's binary_logloss: 0.139776
Training fold 3, hash: 78
Training until validation scores don't improve for 20 rounds.
[100]	training's binary_logloss: 0.105743	valid_1's binary_logloss: 0.139965
Early s

NameError: name 'append' is not defined

In [11]:
# num_boost_round = 300

# # inputs: train_pd, train_targets_inside, and test_pd
# # probably try https://lightgbm.readthedocs.io/en/latest/Python-API.html#lightgbm.cv, but cluster complications
# y_pred_full_x = []
# test_preds = []

# for fold_num, i in enumerate(range(10)):
#     x_trn = df_train_unstack
#     binary_trn = binary
    
#     # create dataset for lightgbm
#     lgb_train_x = lgb.Dataset(x_trn, binary_trn)
# #     lgb_train_y = lgb.Dataset(x_trn, y_trn_y)
# #     lgb_eval_y = lgb.Dataset(x_val, y_val_y, reference=lgb_train_y)
#     params = {
#     'boosting_type': 'goss',
#     'objective': 'binary',
#     'metric': 'binary_logloss',
#     'num_leaves': 63,
#     'learning_rate': 0.05,  # dynamic one below
#     'feature_fraction': 0.9,
#     'verbose': 0,
#     'lambda': 0.1,
#     'num_threads': 3,
#     'seed': 42+fold_num
# #     'histogram_pool_size' : 2048  # to restrict memory usage
# #     'boost_from_average' : False  # as per warning message
# }
    
#     print("Training fold {}, hash: {}".format(fold_num, np.sum(val_index)%999))
    
#     x_trn = df_train_unstack[trn_index]
#     x_val = df_train_unstack[val_index]
    
#     binary_val = binary[val_index]
#     binary_trn = binary[trn_index]
    
#     # create dataset for lightgbm
#     lgb_train_x = lgb.Dataset(x_trn, binary_trn)
#     lgb_eval_x = lgb.Dataset(x_val, binary_val, reference=lgb_train_x)
# #     lgb_train_y = lgb.Dataset(x_trn, y_trn_y)
# #     lgb_eval_y = lgb.Dataset(x_val, y_val_y, reference=lgb_train_y)
    
#     gbm_x = lgb.train(params,
#                     lgb_train_x,
#                     valid_sets=[lgb_train_x, lgb_eval_x],
#                     num_boost_round=num_boost_round,
#                     early_stopping_rounds=20,
#                     learning_rates=lambda iter: 0.1 * (0.995 ** iter),
#                     verbose_eval=100)
    
# #     gbm_y = lgb.train(params,
# #                     lgb_train_y,
# #                     valid_sets=[lgb_train_y, lgb_eval_y],
# #                     num_boost_round=num_boost_round,
# #                     verbose_eval=100)

#     # eval
    
# #     y_pred_ = np.copy(y_pred)
#     y_pred_x = gbm_x.predict(x_val, num_iteration=gbm_x.best_iteration)
#     y_pred_full_x.append(y_pred_x)
    
# #     y_pred_y = gbm_y.predict(x_val, num_iteration=gbm_y.best_iteration)
# #     train_pred_y = gbm_y.predict(x_trn, num_iteration=gbm_y.best_iteration).reshape(trn_index.shape[0], 1)
# #     y_pred_full_y[val_index] = y_pred_y
# #     y_pred_y = y_pred_y.reshape(val_index.shape[0], 1)
    
# #     y_pred = np.concatenate([y_pred_x, y_pred_y], axis=1)
# #     trn_pred = np.concatenate([train_pred_x, train_pred_y], axis=1)
# #     pred_cate = compare(y_pred, binary_val, fold_num)
# #     pred_cate = compare(trn_pred, binary_trn, fold_num)

#     print('The F1-PC-RC-ROC of full prediction is: {:.5f}-{:.5f}-{:.5f}-{:.5f}'
#        .format(*scoring_package(binary_trn, train_pred_x)))
    
#     print('The F1-PC-RC-ROC of full prediction is: {:.5f}-{:.5f}-{:.5f}-{:.5f}'
#        .format(*scoring_package(binary_val, y_pred_x)))
    
#     # testing
# #     df_test["clus_entry"] = test_query_entry[fold_num].ravel()  # different fold have different cluster entries
# #     df_test["clus_exit"] = test_query_exit[fold_num].ravel()
# #     df_test["clus_entry_w"] = test_query_entry_w[fold_num].ravel()  # different fold have different cluster entries
# #     df_test["clus_exit_w"] = test_query_exit_w[fold_num].ravel()
    
#     df_test_unstack = df_test.values
#     df_test_unstack = np.concatenate([df_test_unstack, time_dummy], axis=1)
#     df_test_unstack = df_test_unstack.reshape((int(df_test_unstack.shape[0]/21), 39*21))
#     df_test_unstack = np.concatenate([df_test_unstack, test_embedding], axis=1)
    
#     test_pred_x = gbm_x.predict(df_test_unstack, num_iteration=gbm_x.best_iteration).reshape(df_test_unstack.shape[0], 1)
# #     test_pred_y = gbm_y.predict(df_test_unstack, num_iteration=gbm_y.best_iteration).reshape(df_test_unstack.shape[0], 1)
# #     test_preds.append(np.concatenate([test_pred_x, test_pred_y], axis=1))
#     test_preds.append(test_pred_x)


In [12]:
# pred_cate = np.average(test_preds, axis=0)
# y_pred_full_x = np.average(y_pred_full_x, axis=0)
# np.save("pred_cate_1.npy", pred_cate)
# np.save("valid_pred_1.npy", y_pred_full_x)

In [13]:
# num_boost_round = 600

# # inputs: train_pd, train_targets_inside, and test_pd
# # probably try https://lightgbm.readthedocs.io/en/latest/Python-API.html#lightgbm.cv, but cluster complications
# y_pred_full_x = []
# test_preds = []

# for fold_num, i in enumerate(range(10)):
#     x_trn = df_train_unstack
#     binary_trn = binary
    
#     # create dataset for lightgbm
#     lgb_train_x = lgb.Dataset(x_trn, binary_trn)
# #     lgb_train_y = lgb.Dataset(x_trn, y_trn_y)
# #     lgb_eval_y = lgb.Dataset(x_val, y_val_y, reference=lgb_train_y)
#     params = {
#     'boosting_type': 'dart',
#     'objective': 'binary',
#     'metric': 'binary_logloss',
#     'num_leaves': 63,
#     'learning_rate': 0.1,  # dynamic one below
#     'feature_fraction': 0.9,
#     'verbose': 0,
#     'lambda': 0.1,
#     'num_threads': 3,
#     'seed': 42+fold_num
# #     'histogram_pool_size' : 2048  # to restrict memory usage
# #     'boost_from_average' : False  # as per warning message
# }
    
#     print("Training fold {}, hash: {}".format(fold_num, np.sum(val_index)%999))
    
#     x_trn = df_train_unstack[trn_index]
#     x_val = df_train_unstack[val_index]
    
#     binary_val = binary[val_index]
#     binary_trn = binary[trn_index]
    
#     # create dataset for lightgbm
#     lgb_train_x = lgb.Dataset(x_trn, binary_trn)
#     lgb_eval_x = lgb.Dataset(x_val, binary_val, reference=lgb_train_x)
# #     lgb_train_y = lgb.Dataset(x_trn, y_trn_y)
# #     lgb_eval_y = lgb.Dataset(x_val, y_val_y, reference=lgb_train_y)
    
#     gbm_x = lgb.train(params,
#                     lgb_train_x,
#                     valid_sets=[lgb_train_x, lgb_eval_x],
#                     num_boost_round=num_boost_round,
#                     verbose_eval=100)
    
# #     gbm_y = lgb.train(params,
# #                     lgb_train_y,
# #                     valid_sets=[lgb_train_y, lgb_eval_y],
# #                     num_boost_round=num_boost_round,
# #                     verbose_eval=100)

#     # eval
    
# #     y_pred_ = np.copy(y_pred)
#     y_pred_x = gbm_x.predict(x_val, num_iteration=gbm_x.best_iteration)
#     y_pred_full_x.append(y_pred_x)
    
# #     y_pred_y = gbm_y.predict(x_val, num_iteration=gbm_y.best_iteration)
# #     train_pred_y = gbm_y.predict(x_trn, num_iteration=gbm_y.best_iteration).reshape(trn_index.shape[0], 1)
# #     y_pred_full_y[val_index] = y_pred_y
# #     y_pred_y = y_pred_y.reshape(val_index.shape[0], 1)
    
# #     y_pred = np.concatenate([y_pred_x, y_pred_y], axis=1)
# #     trn_pred = np.concatenate([train_pred_x, train_pred_y], axis=1)
# #     pred_cate = compare(y_pred, binary_val, fold_num)
# #     pred_cate = compare(trn_pred, binary_trn, fold_num)

#     print('The F1-PC-RC-ROC of full prediction is: {:.5f}-{:.5f}-{:.5f}-{:.5f}'
#        .format(*scoring_package(binary_trn, train_pred_x)))
    
#     print('The F1-PC-RC-ROC of full prediction is: {:.5f}-{:.5f}-{:.5f}-{:.5f}'
#        .format(*scoring_package(binary_val, y_pred_x)))
    
#     # testing
# #     df_test["clus_entry"] = test_query_entry[fold_num].ravel()  # different fold have different cluster entries
# #     df_test["clus_exit"] = test_query_exit[fold_num].ravel()
# #     df_test["clus_entry_w"] = test_query_entry_w[fold_num].ravel()  # different fold have different cluster entries
# #     df_test["clus_exit_w"] = test_query_exit_w[fold_num].ravel()
    
#     df_test_unstack = df_test.values
#     df_test_unstack = np.concatenate([df_test_unstack, time_dummy], axis=1)
#     df_test_unstack = df_test_unstack.reshape((int(df_test_unstack.shape[0]/21), 39*21))
#     df_test_unstack = np.concatenate([df_test_unstack, test_embedding], axis=1)
    
#     test_pred_x = gbm_x.predict(df_test_unstack, num_iteration=gbm_x.best_iteration).reshape(df_test_unstack.shape[0], 1)
# #     test_pred_y = gbm_y.predict(df_test_unstack, num_iteration=gbm_y.best_iteration).reshape(df_test_unstack.shape[0], 1)
# #     test_preds.append(np.concatenate([test_pred_x, test_pred_y], axis=1))
#     test_preds.append(test_pred_x)


In [14]:
# pred_cate = np.average(test_preds, axis=0)
# y_pred_full_x = np.average(y_pred_full_x, axis=0)
# np.save("pred_cate_2.npy", pred_cate)
# np.save("valid_pred_2.npy", y_pred_full_x)

# Submission

In [15]:
# test_preds_ = np.concatenate([np.average(test_preds[:, :, 0], axis=0).reshape(test_preds.shape[1], 1), np.average(test_preds[:, :, 1], axis=0).reshape(test_preds.shape[1], 1)], axis=1)
# pred_cate = []
# for i, j in enumerate(test_preds_):
#     incity = whether_in_city(denormalize_x(j[0]), denormalize_y(j[1]))
#     pred_cate.append(incity)
# pred_cate = np.array(pred_cate)

In [16]:
df_submit = pd.read_csv("../input/ey-nextwave/data_test/data_test.csv")
df_submit = df_submit[df_submit['x_exit'].isnull()]
df_submit = df_submit[['trajectory_id']].copy()
df_submit = df_submit.rename(columns = {'trajectory_id':'id'})
df_submit['target'] = pred_cate
df_submit.to_csv('submission.csv', index=False)

In [17]:
df_submit = pd.read_csv("submission.csv")
df_submit.head()

,id,target
0,traj_00032f51796fd5437b238e3a9823d13d_31_5,0.007
1,traj_000479418b5561ab694a2870cc04fd43_25_10,0.006
2,traj_000506a39775e5bca661ac80e3f466eb_29_5,0.995
3,traj_0005401ceddaf27a9b7f0d42ef1fbe95_1_4,0.021
4,traj_00063a4f6c12e1e4de7d876580620667_3_4,0.081


In [18]:
df_submit.tail()

,id,target
33510,traj_ffe98f6e0adf12f9c7b51c4e9607a87a_15_13,0.001
33511,traj_fff607ecd3f8d3dcb65791e8b4c22a5f_3_25,0.581
33512,traj_fff813b56230c2f026f783f5b9f9ca90_19_0,0.002
33513,traj_fff9400843a88c3bfe52e7ce8bf97316_19_17,0.023
33514,traj_fff9552047b095e8242b4913f3289a26_25_7,0.001
